In [1]:
from kafka import KafkaConsumer
from pyspark import SparkConf, SparkContext, SQLContext, HiveContext
from pyspark.sql import SparkSession
from os.path import abspath
import numpy as np
import re

In [21]:
pip install findspark

  Using cached findspark-2.0.1-py2.py3-none-any.whl.metadata (352 bytes)
Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)
Note: you may need to restart the kernel to use updated packages.


In [113]:
!hdfs dfs -ls /

Found 2 items
drwxr-xr-x   - roser supergroup          0 2025-04-02 18:20 /TFM
drwxr-xr-x   - roser supergroup          0 2025-04-02 12:54 /temp


In [2]:
!hdfs dfs -ls /TFM/

Found 4 items
drwxr-xr-x   - roser supergroup          0 2025-04-02 14:11 /TFM/old_news
-rw-r--r--   1 roser supergroup        108 2025-04-02 18:13 /TFM/pais-us.csv
-rw-r--r--   1 roser supergroup        992 2025-04-02 13:26 /TFM/paisos-ue.csv
drwxr-xr-x   - roser supergroup          0 2025-04-02 18:20 /TFM/president


In [57]:
!hdfs dfs -rm /TFM/pais-us.csv

Deleted /TFM/pais-us.csv


In [58]:
# Copy file from local to hdfs.
!hdfs dfs -put /home/$USER/TFM/data/pais-us.csv  /TFM
!hdfs dfs -put /home/$USER/TFM/data/paisos-ue.csv  /TFM

In [11]:
!hdfs dfs -ls /TFM/

Found 2 items
-rw-r--r--   1 roser supergroup         84 2025-04-02 13:17 /TFM/pais-us.csv
-rw-r--r--   1 roser supergroup        992 2025-04-02 13:26 /TFM/paisos-ue.csv


In [59]:
# Check if there ara corruptet block.
print("\n## Report to check for inconsistencies:\n")
!hdfs fsck /TFM/pais-us.csv


## Report to check for inconsistencies:

Connecting to namenode via http://0.0.0.0:9870/fsck?ugi=roser&path=%2FTFM%2Fpais-us.csv
FSCK started by roser (auth:SIMPLE) from /127.0.0.1 for path /TFM/pais-us.csv at Wed Apr 02 18:13:51 CEST 2025


Status: HEALTHY
 Number of data-nodes:	1
 Number of racks:		1
 Total dirs:			0
 Total symlinks:		0

Replicated Blocks:
 Total size:	108 B
 Total files:	1
 Total blocks (validated):	1 (avg. block size 108 B)
 Minimally replicated blocks:	1 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	0 (0.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	1
 Average block replication:	1.0
 Missing blocks:		0
 Corrupt blocks:		0
 Missing replicas:		0 (0.0 %)
 Blocks queued for replication:	0

Erasure Coded Block Groups:
 Total size:	0 B
 Total files:	0
 Total block groups (validated):	0
 Minimally erasure-coded block groups:	0
 Over-erasure-coded block groups:	0
 Under-erasure-coded block groups:	0
 Unsatisfactory placemen

In [60]:
# Show content head.
print("\n## Sample contenet file:\n")
!hdfs dfs -cat /TFM/pais-us.csv | head -10


## Sample contenet file:

"Nom","Name","Codi ISO","Codi ISO Llengua"
"Estats Units d'Amèrica","United States of America","US","en"


In [3]:
conf = SparkConf()
conf.setMaster("local[1]")
conf.setAppName("Consumer_Hive")
sc = SparkContext.getOrCreate(conf=conf) 
print(sc.version)

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


3.5.5


In [9]:
# Iniciem SQLContext
sqlContext = SQLContext(sc)

# Carreguem el CSV en un dataframe
news_csv = sqlContext.read.csv("/TFM/pais-us.csv", header=True, inferSchema=True, sep=',')

# fem un print del count de tweets
print("\n## Loaded dataset contains %d news:\n" % news_csv.count())

# mostrem el esquema del dataframe
news_csv.printSchema()

/home/roser/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(



## Loaded dataset contains 1 news:

root
 |-- Nom: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Codi ISO: string (nullable = true)
 |-- Codi ISO Llengua: string (nullable = true)



In [63]:
news_csv.show()

+--------------------+--------------------+--------+----------------+
|                 Nom|                Name|Codi ISO|Codi ISO Llengua|
+--------------------+--------------------+--------+----------------+
|Estats Units d'Am...|United States of ...|      US|              en|
+--------------------+--------------------+--------+----------------+



In [19]:
!hdfs dfs -put /home/$USER/TFM/data/old_news  /TFM/old_news

In [7]:
!hdfs dfs -ls /TFM/old_news

Found 651 items
-rw-r--r--   1 roser supergroup    2544177 2025-04-02 14:10 /TFM/old_news/1970_10_news_us_old.csv
-rw-r--r--   1 roser supergroup    2352088 2025-04-02 14:10 /TFM/old_news/1970_11_news_us_old.csv
-rw-r--r--   1 roser supergroup    2323784 2025-04-02 14:10 /TFM/old_news/1970_12_news_us_old.csv
-rw-r--r--   1 roser supergroup    2539819 2025-04-02 14:10 /TFM/old_news/1970_1_news_us_old.csv
-rw-r--r--   1 roser supergroup    2269022 2025-04-02 14:11 /TFM/old_news/1970_2_news_us_old.csv
-rw-r--r--   1 roser supergroup    2533818 2025-04-02 14:10 /TFM/old_news/1970_3_news_us_old.csv
-rw-r--r--   1 roser supergroup    2280036 2025-04-02 14:10 /TFM/old_news/1970_4_news_us_old.csv
-rw-r--r--   1 roser supergroup    2469840 2025-04-02 14:10 /TFM/old_news/1970_5_news_us_old.csv
-rw-r--r--   1 roser supergroup    2395103 2025-04-02 14:11 /TFM/old_news/1970_6_news_us_old.csv
-rw-r--r--   1 roser supergroup    2192998 2025-04-02 14:11 /TFM/old_news/1970_7_news_us_old.csv
-rw-r--r-- 

In [21]:
!hdfs dfs -cat /TFM/old_news/2025_3_news_us_old.csv | head -10

iso_code,title,description,pubDate
us,A Day of American Infamy,"In August 1941, about four months before the Japanese attack on Pearl Harbor, Franklin Roosevelt met with Winston Churchill aboard warships in Newfoundlands Placentia Bay and agreed to the Atlantic Charter, a joint declaration by the worlds leading democratic powers on common principles for a postwar world.",2025-03-01T00:00:33+0000
us,Behind the Collision: Trump Jettisons Ukraine on His Way to a Larger Goal,"After five weeks in which President Trump made clear his determination to scrap Americas traditional sources of power  its alliances among like-minded democracies  and return the country to an era of raw great-power negotiations, he left one question hanging: How far would he go in sacrificing Ukraine to his vision?",2025-03-01T00:15:42+0000
us,"Despite Efforts to Save It, a Beloved Pink House in Massachusetts Will Be Razed","A battered pink house on the coast of Massachusetts, beloved by generations of artists and ro

In [22]:
print(sc.version)

3.5.5


In [5]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

# Create my_spark
my_spark = SparkSession.builder.getOrCreate()

# Print my_spark
print(my_spark)


your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [17]:
df=my_spark.sql("show databases")
df.show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [11]:
my_spark.sql("CREATE DATABASE IF NOT EXISTS tfm ")
print(my_spark.catalog.listDatabases())
my_spark.sql("USE tfm")

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/home/roser/TFM/spark-warehouse'), Database(name='tfm', catalog='spark_catalog', description='', locationUri='file:/home/roser/TFM/spark-warehouse/tfm.db')]


DataFrame[]

In [67]:
my_spark.sql("DROP TABLE IF EXISTS paisos_iso")
my_spark.sql("DROP TABLE IF EXISTS pais_iso")

# Print the tables in the catalog
print(my_spark.catalog.listTables())

[]


In [68]:
iso_us = sqlContext.read.csv("/TFM/pais-us.csv", header=True, inferSchema=True, sep=',')
iso_ue = sqlContext.read.csv("/TFM/paisos-ue.csv", header=True, inferSchema=True, sep=',')
iso_ue.write.saveAsTable("paisos_iso")
iso_us.write.mode('append').saveAsTable("paisos_iso")

In [69]:
table_ue=my_spark.sql("select * from paisos_iso")
table_ue.show(n=1000, truncate=False)

+----------------------+------------------------+--------+----------------+
|Nom                   |Name                    |Codi ISO|Codi ISO Llengua|
+----------------------+------------------------+--------+----------------+
|Alemanya              |Germany                 |DE      |de              |
|Àustria               |Austria                 |AT      |de              |
|Bèlgica               |Belgium                 |BE      |nl,fr,de        |
|Bulgària              |Bulgaria                |BG      |bg              |
|Xipre                 |Cyprus                  |CY      |el,tr           |
|Croàcia               |Croatia                 |HR      |hr              |
|Dinamarca             |Denmark                 |DK      |da              |
|Eslovàquia            |Slovakia                |SK      |sk              |
|Eslovènia             |Slovenia                |SI      |sl              |
|Espanya               |Spain                   |ES      |es              |
|Estònia    

In [12]:
print(table_ue.count())

NameError: name 'table_ue' is not defined

In [53]:
iso_us.show()

+--------------------+--------------------+--------+
|                 Nom|                Name|Codi ISO|
+--------------------+--------------------+--------+
|Estats Units d'Am...|United States of ...|      US|
+--------------------+--------------------+--------+



In [71]:
# copy presidents fom locat to hdfs
!hdfs dfs -put /home/$USER/TFM/data/president  /TFM/president

In [8]:
!hdfs dfs -ls /TFM/president | awk '{print $NF}'

items
/TFM/president/presidents-at.csv
/TFM/president/presidents-be.csv
/TFM/president/presidents-bg.csv
/TFM/president/presidents-cy.csv
/TFM/president/presidents-cz.csv
/TFM/president/presidents-de.csv
/TFM/president/presidents-dk.csv
/TFM/president/presidents-ee.csv
/TFM/president/presidents-es.csv
/TFM/president/presidents-fi.csv
/TFM/president/presidents-fr.csv
/TFM/president/presidents-gb.csv
/TFM/president/presidents-gr.csv
/TFM/president/presidents-hr.csv
/TFM/president/presidents-hu.csv
/TFM/president/presidents-ie.csv
/TFM/president/presidents-it.csv
/TFM/president/presidents-lt.csv
/TFM/president/presidents-lu.csv
/TFM/president/presidents-lv.csv
/TFM/president/presidents-mt.csv
/TFM/president/presidents-nl.csv
/TFM/president/presidents-pl.csv
/TFM/president/presidents-pt.csv
/TFM/president/presidents-ro.csv
/TFM/president/presidents-se.csv
/TFM/president/presidents-si.csv
/TFM/president/presidents-sk.csv
/TFM/president/presidents-turquia.csv
/TFM/president/presidents-us.csv

In [ ]:
list_file_presi = !hdfs dfs -ls /TFM/president | awk '{print $NF}'
list_file_presi = list_file_presi[1:]
print(list_file_presi)


['/TFM/president/presidents-at.csv', '/TFM/president/presidents-be.csv', '/TFM/president/presidents-bg.csv', '/TFM/president/presidents-cy.csv', '/TFM/president/presidents-cz.csv', '/TFM/president/presidents-de.csv', '/TFM/president/presidents-dk.csv', '/TFM/president/presidents-ee.csv', '/TFM/president/presidents-es.csv', '/TFM/president/presidents-fi.csv', '/TFM/president/presidents-fr.csv', '/TFM/president/presidents-gb.csv', '/TFM/president/presidents-gr.csv', '/TFM/president/presidents-hr.csv', '/TFM/president/presidents-hu.csv', '/TFM/president/presidents-ie.csv', '/TFM/president/presidents-it.csv', '/TFM/president/presidents-lt.csv', '/TFM/president/presidents-lu.csv', '/TFM/president/presidents-lv.csv', '/TFM/president/presidents-mt.csv', '/TFM/president/presidents-nl.csv', '/TFM/president/presidents-pl.csv', '/TFM/president/presidents-pt.csv', '/TFM/president/presidents-ro.csv', '/TFM/president/presidents-se.csv', '/TFM/president/presidents-si.csv', '/TFM/president/presidents-

In [ ]:
# Load csv de presidents paisos com taules
for file in list_file_presi:
    name_file = np.array(file.split("/"))[-1][:-4]
    name_file = re.sub(r'-', '_', name_file)
    file_presis = sqlContext.read.csv(file, header=True, inferSchema=True, sep=',')
    file_presis.write.saveAsTable(name_file)
 

In [14]:
print(my_spark.catalog.listTables())   

[]


In [13]:
table_ue=my_spark.sql("select * from presidents_turquia")
table_ue.show(n=1000, truncate=False)

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `presidents_turquia` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [presidents_turquia], [], false


In [15]:
# Passar noticies descarregades diariament per començar, despres s'aniran afe
# afegint desde el py del consumer.
!hdfs dfs -put /home/$USER/TFM/data/news  /TFM/news

In [27]:
!hdfs dfs -ls  /TFM/news/news_r*.csv

-rw-r--r--   1 roser supergroup     294748 2025-04-03 17:24 /TFM/news/news_ro.csv


In [58]:
!hdfs dfs -put /home/$USER/TFM/data/news/news_us.csv /TFM/news/news_us/part-00010.csv

In [34]:
!hdfs dfs -rm /TFM/news/news_ro.csv

Deleted /TFM/news/news_ro.csv


In [15]:
df_ro = my_spark.read.csv("/TFM/news/news_us/", header=True, inferSchema=True)
df_ro.show()  # Mostra les primeres files

NameError: name 'my_spark' is not defined

In [60]:
df_ro.count()

430

In [56]:
df_ro = my_spark.read.csv("/TFM/news/news_es/", header=True, inferSchema=True)
df_ro.count()  # Mostra les primeres files

292

In [ ]:
file = "/TFM/news/news_be.csv"
name_dir = "/TFM/news/news_be"
!hdfs dfs -cp {file} {name_dir}

In [47]:
!hdfs dfs -rm {file}

Deleted /TFM/news/news_be.csv


In [52]:
list_file_news = !hdfs dfs -ls /TFM/news/news*.csv | awk '{print $NF}'
list_file_news = list_file_news[1:]
print(list_file_news)

# Load csv de news as part
for file in list_file_news:
    name_dir = file[:-4]+'/part-00010.csv'
    !hdfs dfs -cp {file} {name_dir}
    !hdfs dfs -rm {file}


['/TFM/news/news_bg.csv', '/TFM/news/news_cy.csv', '/TFM/news/news_cz.csv', '/TFM/news/news_de.csv', '/TFM/news/news_dk.csv', '/TFM/news/news_ee.csv', '/TFM/news/news_en.csv', '/TFM/news/news_es.csv', '/TFM/news/news_fi.csv', '/TFM/news/news_fr.csv', '/TFM/news/news_gb.csv', '/TFM/news/news_gr.csv', '/TFM/news/news_hr.csv', '/TFM/news/news_hu.csv', '/TFM/news/news_ie.csv', '/TFM/news/news_it.csv', '/TFM/news/news_lt.csv', '/TFM/news/news_lu.csv', '/TFM/news/news_lv.csv', '/TFM/news/news_mt.csv', '/TFM/news/news_nl.csv', '/TFM/news/news_pl.csv', '/TFM/news/news_pt.csv', '/TFM/news/news_se.csv', '/TFM/news/news_si.csv', '/TFM/news/news_sk.csv', '/TFM/news/news_us.csv']
Deleted /TFM/news/news_bg.csv
Deleted /TFM/news/news_cy.csv
Deleted /TFM/news/news_cz.csv
Deleted /TFM/news/news_de.csv
Deleted /TFM/news/news_dk.csv
Deleted /TFM/news/news_ee.csv
cp: `/TFM/news/news_en/part-00010.csv': No such file or directory: `hdfs://0.0.0.0:9000/TFM/news/news_en/part-00010.csv'
Deleted /TFM/news/news_

In [61]:
!hdfs dfs -ls  /TFM/

Found 5 items
drwxr-xr-x   - roser supergroup          0 2025-04-04 10:03 /TFM/news
drwxr-xr-x   - roser supergroup          0 2025-04-02 14:11 /TFM/old_news
-rw-r--r--   1 roser supergroup        108 2025-04-02 18:13 /TFM/pais-us.csv
-rw-r--r--   1 roser supergroup        992 2025-04-02 13:26 /TFM/paisos-ue.csv
drwxr-xr-x   - roser supergroup          0 2025-04-02 18:20 /TFM/president


In [62]:
!hdfs dfs -ls  /TFM/news

Found 30 items
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:06 /TFM/news/news_at
-rw-r--r--   1 roser supergroup     182210 2025-04-03 17:25 /TFM/news/news_at.csv
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:42 /TFM/news/news_be
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:47 /TFM/news/news_bg
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:47 /TFM/news/news_cy
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:47 /TFM/news/news_cz
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:47 /TFM/news/news_de
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:47 /TFM/news/news_dk
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:48 /TFM/news/news_ee
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:48 /TFM/news/news_es
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:48 /TFM/news/news_fi
drwxr-xr-x   - roser supergroup          0 2025-04-04 09:48 /TFM/news/news_fr
drwxr-xr-x   - roser supergroup          0 20

In [ ]:
# Check data old news (timestamp)
f_old_news = my_spark.read.parquet("/TFM/old_news", header=True, inferSchema=True)
f_old_news.show()  # Mostra les primeres files

+--------+--------------------+--------------------+--------------------+
|iso_code|               title|         description|             pubDate|
+--------+--------------------+--------------------+--------------------+
|      us|Likes Taking Risk...|EVAN SHEFTEL of N...|2008-10-01T00:01:...|
|      us|For Oktoberfest, ...|I MUST confess th...|2008-10-01T00:03:...|
|      us|Tighter Credit On...|DETROIT  After en...|2008-10-01T00:04:...|
|      us|And Then There Wa...|SINCE his death l...|2008-10-01T00:06:...|
|      us|At the Parker Mer...|The name of the n...|2008-10-01T00:09:...|
|      us|Awaiting Palins P...|Those waiting for...|2008-10-01T00:11:...|
|      us|Tunisian Couscous...|Hand-rolled cousc...|2008-10-01T00:11:...|
|      us|David Jones, Film...|David Jones, a ce...|2008-10-01T00:12:...|
|      us|Serious Tool Budd...|These colorful si...|2008-10-01T00:12:...|
|      us|   Names of the Dead|The Department of...|2008-10-01T00:12:...|
|      us|Swiss Said to Be ...|The vei

In [74]:
f_old_news.schema

StructType([StructField('iso_code', StringType(), True), StructField('title', StringType(), True), StructField('description', StringType(), True), StructField('lang_tranlation', StringType(), True), StructField('title_translated', StringType(), True), StructField('description_translated', StringType(), True), StructField('pubDate', StringType(), True)])

In [69]:
from pyspark.sql.functions import to_timestamp
f_old_news = f_old_news.withColumn("pubDate", to_timestamp(f_old_news["pubDate"], "yyyy-MM-dd'T'HH:mm:ss'Z'"))

In [70]:
f_old_news.schema

StructType([StructField('iso_code', StringType(), True), StructField('title', StringType(), True), StructField('description', StringType(), True), StructField('pubDate', TimestampType(), True)])

In [2]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [ ]:
# Inicia Spark NLP
spark_nlp = sparknlp.start()

pipeline = PretrainedPipeline('explain_document_ml', lang='en')
 

your 131072x1 screen size is bogus. expect trouble


:: loading settings :: url = jar:file:/home/roser/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/roser/.ivy2/cache
The jars for the packages stored in: /home/roser/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-14b2985f-745a-410e-a702-c94bc72ace37;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.5.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central


explain_document_ml download started this may take some time.
Approx size to download 9 MB
[ | ]

25/04/04 11:36:42 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/04/04 11:36:42 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


explain_document_ml download started this may take some time.
Approximate size to download 9 MB
[ / ]Download done! Loading the resource.
[ \ ]

[ | ]

[ — ]

[OK!]


In [122]:

news_text = "The Supreme Court heard arguments on\xa0whether South Carolina can't remove Planned Parenthood clinics from its state Medicaid program, even though those funds cannot generally be used to fund abortions."
pip_dict = pipeline.annotate(news_text)
for key, value in pip_dict.items():
    print(f"{key}: {value}")

document: ["The Supreme Court heard arguments on\xa0whether South Carolina can't remove Planned Parenthood clinics from its state Medicaid program, even though those funds cannot generally be used to fund abortions."]
spell: ['The', 'Supreme', 'Court', 'heard', 'arguments', 'on\xa0whether', 'South', 'Carolina', "can't", 'remove', 'Planned', 'Parenthood', 'clinics', 'from', 'its', 'state', 'Medicaid', 'program', ',', 'even', 'though', 'those', 'funds', 'cannot', 'generally', 'be', 'used', 'to', 'fund', 'abortions', '.']
pos: ['DT', 'NNP', 'NNP', 'VBD', 'NNS', 'RB', 'NNP', 'NNP', 'VBD', 'VB', 'VBN', 'NNP', 'NNS', 'IN', 'PRP$', 'NN', 'NNP', 'NN', ',', 'RB', 'IN', 'DT', 'NNS', 'NN', 'RB', 'VB', 'VBN', 'TO', 'VB', 'NNS', '.']
lemmas: ['The', 'Supreme', 'Court', 'hear', 'argument', 'on\xa0whether', 'South', 'Carolina', "can't", 'remove', 'Planned', 'Parenthood', 'clinic', 'from', 'it', 'state', 'Medicaid', 'program', ',', 'even', 'though', 'those', 'fund', 'cannot', 'generally', 'be', 'use',

In [37]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

schema_news = StructType([
    StructField("iso_code", StringType(), True),
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("lang_tranlation", StringType(), True),
    StructField("title_translated", StringType(), False),
    StructField("description_translated", StringType(), True),
    StructField("pubDate", TimestampType(), False)
])

# Check data old news (timestamp)
f_old_news = my_spark.read.csv("/TFM/news/news_es", header=False, schema=schema_news)
f_old_news.show(5)  # Mostra les primeres files
f_old_news.printSchema()

+--------+--------------------+--------------------+--------------------+--------------------+----------------------+-------------------+
|iso_code|               title|         description|     lang_tranlation|    title_translated|description_translated|            pubDate|
+--------+--------------------+--------------------+--------------------+--------------------+----------------------+-------------------+
|iso_code|               title|         description|     lang_tranlation|    title_translated|  description_trans...|               NULL|
|      es|El Sala Ourense s...|El Sala Ourense c...|                  en|The Ourense room ...|  The Ourense room ...|2025-03-18 05:04:00|
|      es|"El juguete que t...|"El Furby fue uno...| que recordaba a ...| lo hizo irresist...|   se vendieron 40 ...|               NULL|
|      es|El dineral que ga...|Más concretament...|                  en|The money that Il...|  More specifically...|2025-03-18 05:30:00|
|      es|Aitana y la perve...|Ant

In [ ]:

schema_old_news = StructType([
    StructField("iso_code", StringType(), True),
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("pubDate", TimestampType(), True)
])

# Check data old news (timestamp)
f_old_news = my_spark.read.csv("/TFM/old_news", header=False, schema=schema_old_news)
f_old_news.show()  # Mostra les primeres files

+--------+--------------------+--------------------+-------------------+
|iso_code|               title|         description|            pubDate|
+--------+--------------------+--------------------+-------------------+
|iso_code|               title|         description|               NULL|
|      us|Likes Taking Risk...|EVAN SHEFTEL of N...|2008-10-01 02:01:06|
|      us|For Oktoberfest, ...|I MUST confess th...|2008-10-01 02:03:26|
|      us|Tighter Credit On...|DETROIT  After en...|2008-10-01 02:04:40|
|      us|And Then There Wa...|SINCE his death l...|2008-10-01 02:06:24|
|      us|At the Parker Mer...|The name of the n...|2008-10-01 02:09:15|
|      us|Awaiting Palins P...|Those waiting for...|2008-10-01 02:11:01|
|      us|Tunisian Couscous...|Hand-rolled cousc...|2008-10-01 02:11:39|
|      us|David Jones, Film...|David Jones, a ce...|2008-10-01 02:12:25|
|      us|Serious Tool Budd...|These colorful si...|2008-10-01 02:12:25|
|      us|   Names of the Dead|The Department of...

In [40]:
!hdfs dfs -ls /TFM/news/news_at

Found 22 items
-rw-r--r--   1 roser supergroup          0 2025-04-04 10:35 /TFM/news/news_at/_SUCCESS
-rw-r--r--   1 roser supergroup          0 2025-04-04 09:02 /TFM/news/news_at/part-00000-13c0b900-0883-4267-8081-e263e6925e30-c000.csv
-rw-r--r--   1 roser supergroup          0 2025-04-04 09:06 /TFM/news/news_at/part-00000-201b73c1-15da-42f5-b442-8448b4506333-c000.csv
-rw-r--r--   1 roser supergroup          0 2025-04-04 09:06 /TFM/news/news_at/part-00000-4665845c-05b0-44d5-b078-13db5f8ff394-c000.csv
-rw-r--r--   1 roser supergroup          0 2025-04-04 09:06 /TFM/news/news_at/part-00000-5c4296fd-06ad-4dc8-b4d0-fbd0226012db-c000.csv
-rw-r--r--   1 roser supergroup       7779 2025-04-04 10:35 /TFM/news/news_at/part-00000-97bb5ed3-ac35-471a-9f22-5f439850a217-c000.csv
-rw-r--r--   1 roser supergroup          0 2025-04-04 09:06 /TFM/news/news_at/part-00000-9e9e4fba-08dc-426b-93d3-d2126df6d458-c000.csv
-rw-r--r--   1 roser supergroup          0 2025-04-04 09:06 /TFM/news/news_at/part-00000

In [41]:
!hdfs dfs -rm -r /TFM/news/news_*/*

Deleted /TFM/news/news_at/_SUCCESS
Deleted /TFM/news/news_at/part-00000-13c0b900-0883-4267-8081-e263e6925e30-c000.csv
Deleted /TFM/news/news_at/part-00000-201b73c1-15da-42f5-b442-8448b4506333-c000.csv
Deleted /TFM/news/news_at/part-00000-4665845c-05b0-44d5-b078-13db5f8ff394-c000.csv
Deleted /TFM/news/news_at/part-00000-5c4296fd-06ad-4dc8-b4d0-fbd0226012db-c000.csv
Deleted /TFM/news/news_at/part-00000-97bb5ed3-ac35-471a-9f22-5f439850a217-c000.csv
Deleted /TFM/news/news_at/part-00000-9e9e4fba-08dc-426b-93d3-d2126df6d458-c000.csv
Deleted /TFM/news/news_at/part-00000-a5d7c781-2fed-4d6c-8b6c-226cfe27189c-c000.csv
Deleted /TFM/news/news_at/part-00000-af31da25-324e-4567-966c-6a2ffe5f4891-c000.csv
Deleted /TFM/news/news_at/part-00000-c928f34b-82bc-42d0-8c3a-580f8f3ef49a-c000.csv
Deleted /TFM/news/news_at/part-00001-97bb5ed3-ac35-471a-9f22-5f439850a217-c000.csv
Deleted /TFM/news/news_at/part-00001-9e9e4fba-08dc-426b-93d3-d2126df6d458-c000.csv
Deleted /TFM/news/news_at/part-00001-c928f34b-82bc-4

In [63]:
!ls /home/$USER/TFM/data/old_news

1970_10_news_us_old.csv  1988_11_news_us_old.csv  2006_12_news_us_old.csv
1970_11_news_us_old.csv  1988_12_news_us_old.csv  2006_1_news_us_old.csv
1970_12_news_us_old.csv  1988_1_news_us_old.csv   2006_2_news_us_old.csv
1970_1_news_us_old.csv	 1988_2_news_us_old.csv   2006_3_news_us_old.csv
1970_2_news_us_old.csv	 1988_3_news_us_old.csv   2006_4_news_us_old.csv
1970_3_news_us_old.csv	 1988_4_news_us_old.csv   2006_5_news_us_old.csv
1970_4_news_us_old.csv	 1988_5_news_us_old.csv   2006_6_news_us_old.csv
1970_5_news_us_old.csv	 1988_6_news_us_old.csv   2006_7_news_us_old.csv
1970_6_news_us_old.csv	 1988_7_news_us_old.csv   2006_8_news_us_old.csv
1970_7_news_us_old.csv	 1988_8_news_us_old.csv   2006_9_news_us_old.csv
1970_8_news_us_old.csv	 1988_9_news_us_old.csv   2007_10_news_us_old.csv
1970_9_news_us_old.csv	 1989_10_news_us_old.csv  2007_11_news_us_old.csv
1971_10_news_us_old.csv  1989_11_news_us_old.csv  2007_12_news_us_old.csv
1971_11_news_us_old.csv  1989_12_news_us_old.csv  2007_1

In [ ]:
from pyspark.sql.functions import col

# Carregar noticies europees diaries a hdfs com a parquet a cada directori de pais
list_file_news = !ls /home/$USER/TFM/data/news | awk '{print $NF}'
list_file_news = list_file_news[1:]
print(list_file_news)

# Load csv de news as parquet
for file in list_file_news:
    name_dir = file[:-4]
    df = my_spark.read.option("header", "true").schema(schema_news).csv("file:///home/roser/TFM/data/news/"+file)
    df_clean = df.filter(col("title_translated").isNotNull() & col("pubDate").isNotNull())
    df.write.mode("append").parquet("/TFM/news/"+name_dir)
    
    

['news_be.csv', 'news_bg.csv', 'news_cy.csv', 'news_cz.csv', 'news_de.csv', 'news_dk.csv', 'news_ee.csv', 'news_es.csv', 'news_fi.csv', 'news_fr.csv', 'news_gb.csv', 'news_gr.csv', 'news_hr.csv', 'news_hu.csv', 'news_ie.csv', 'news_it.csv', 'news_lt.csv', 'news_lu.csv', 'news_lv.csv', 'news_mt.csv', 'news_nl.csv', 'news_pl.csv', 'news_pt.csv', 'news_ro.csv', 'news_se.csv', 'news_si.csv', 'news_sk.csv', 'news_us.csv']


In [61]:
# Check data old news (timestamp)
f_old_news = my_spark.read.parquet("/TFM/news/news_es")
print(f_old_news.count())  # Mostra les primeres files
f_old_news.printSchema()

322
root
 |-- iso_code: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- lang_tranlation: string (nullable = true)
 |-- title_translated: string (nullable = true)
 |-- description_translated: string (nullable = true)
 |-- pubDate: timestamp (nullable = true)



In [1]:
# Eliminar fitxerts parquet
!hdfs dfs -rm -r /TFM/old_news/*

Deleted /TFM/old_news/_SUCCESS
Deleted /TFM/old_news/part-00000-008c1409-2f3d-4423-8476-521436ba1fd2-c000.snappy.parquet
Deleted /TFM/old_news/part-00000-00b6891d-4566-419a-a68c-41db5e001862-c000.snappy.parquet
Deleted /TFM/old_news/part-00000-00b90e1f-d92d-46a5-ad4e-90efea91c5a6-c000.snappy.parquet
Deleted /TFM/old_news/part-00000-01160f2b-fe80-47aa-b591-32d94f28e0d7-c000.snappy.parquet
Deleted /TFM/old_news/part-00000-013395d5-35f9-4057-b538-af98dfb8a4fb-c000.snappy.parquet
Deleted /TFM/old_news/part-00000-01a8a271-8e53-478f-9b17-5ee43adbdccb-c000.snappy.parquet
Deleted /TFM/old_news/part-00000-01f2d7ef-67b8-41ce-b6d3-14bf29ab4795-c000.snappy.parquet
Deleted /TFM/old_news/part-00000-01fa1efa-9edf-400c-8c1d-978859575d9d-c000.snappy.parquet
Deleted /TFM/old_news/part-00000-02584c64-6b1a-4e48-8ad5-fac8c9d0badd-c000.snappy.parquet
Deleted /TFM/old_news/part-00000-028251f4-e889-4048-bc9b-bd91198084e7-c000.snappy.parquet
Deleted /TFM/old_news/part-00000-02a040e7-fc97-4957-87f7-4e28c9350d44

In [7]:
# Validar dir empty
!hdfs dfs -ls /TFM/old_news/

In [ ]:
from pyspark.sql.functions import col, to_date, to_timestamp
from pyspark.sql.types import StructType, StructField, StringType

# Carregar noticies antigues New York Times  a hdfs com a parquet al directori de noticies antigues
list_file_news = !ls /home/$USER/TFM/data/old_news | awk '{print $NF}'
list_file_news = list_file_news[1:]

schema_old_news = StructType([
    StructField("iso_code", StringType(), True),
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("pubDate", StringType(), True)
])

# Load csv de old news as parquet
for file in list_file_news:
    print(file)
   
    df = my_spark.read.option("header", "true").schema(schema_old_news).csv("file:///home/roser/TFM/data/old_news/"+file)

    # to check format data to avoid errors data old 1900-01-01 convertim a timestamp i despres ens quedem sols amb la data
    df = df.withColumn("pubDate", to_timestamp(col("pubDate"), "yyyy-MM-dd'T'HH:mm:ssZ"))
    df = df.withColumn("pubDate", to_date(col("pubDate")))  # només la part de la data
    # Filtrar les dates mal formades
    df_filtered = df.filter(col("pubDate").isNotNull())
    df_filtered.write.mode("append").parquet("/TFM/old_news/")

1970_11_news_us_old.csv


1970_12_news_us_old.csv
1970_1_news_us_old.csv


1970_2_news_us_old.csv
1970_3_news_us_old.csv


1970_4_news_us_old.csv


1970_5_news_us_old.csv
1970_6_news_us_old.csv
1970_7_news_us_old.csv
1970_8_news_us_old.csv
1970_9_news_us_old.csv
1971_10_news_us_old.csv
1971_11_news_us_old.csv
1971_12_news_us_old.csv
1971_1_news_us_old.csv
1971_2_news_us_old.csv
1971_3_news_us_old.csv
1971_4_news_us_old.csv
1971_5_news_us_old.csv
1971_6_news_us_old.csv
1971_7_news_us_old.csv
1971_8_news_us_old.csv
1971_9_news_us_old.csv


...

2017_5_news_us_old.csv


2017_6_news_us_old.csv
2017_7_news_us_old.csv


2017_8_news_us_old.csv
2017_9_news_us_old.csv
2018_10_news_us_old.csv
2018_11_news_us_old.csv
2018_12_news_us_old.csv
2018_1_news_us_old.csv
2018_2_news_us_old.csv
2018_3_news_us_old.csv
2018_4_news_us_old.csv
2018_5_news_us_old.csv
2018_6_news_us_old.csv
2018_7_news_us_old.csv
2018_8_news_us_old.csv
2018_9_news_us_old.csv
2019_10_news_us_old.csv
2019_11_news_us_old.csv
2019_12_news_us_old.csv
2019_1_news_us_old.csv
2019_2_news_us_old.csv
2019_3_news_us_old.csv
2019_4_news_us_old.csv
2019_5_news_us_old.csv
2019_6_news_us_old.csv
2019_7_news_us_old.csv
2019_8_news_us_old.csv
2019_9_news_us_old.csv
2020_10_news_us_old.csv
2020_11_news_us_old.csv


2020_12_news_us_old.csv
2020_1_news_us_old.csv
2020_2_news_us_old.csv
2020_3_news_us_old.csv
2020_4_news_us_old.csv
2020_5_news_us_old.csv
2020_6_news_us_old.csv
2020_7_news_us_old.csv
2020_8_news_us_old.csv
2020_9_news_us_old.csv
2021_10_news_us_old.csv
2021_11_news_us_old.csv


2021_12_news_us_old.csv
2021_1_news_us_old.csv
2021_2_news_us_old.csv
2021_3_news_us_old.csv
2021_4_news_us_old.csv
2021_5_news_us_old.csv
2021_6_news_us_old.csv
2021_7_news_us_old.csv
2021_8_news_us_old.csv
2021_9_news_us_old.csv
2022_10_news_us_old.csv
2022_11_news_us_old.csv
2022_12_news_us_old.csv
2022_1_news_us_old.csv
2022_2_news_us_old.csv
2022_3_news_us_old.csv
2022_4_news_us_old.csv
2022_5_news_us_old.csv
2022_6_news_us_old.csv
2022_7_news_us_old.csv
2022_8_news_us_old.csv
2022_9_news_us_old.csv
2023_10_news_us_old.csv
2023_11_news_us_old.csv
2023_12_news_us_old.csv
2023_1_news_us_old.csv
2023_2_news_us_old.csv
2023_3_news_us_old.csv
2023_4_news_us_old.csv
2023_5_news_us_old.csv
2023_6_news_us_old.csv
2023_7_news_us_old.csv
2023_8_news_us_old.csv
2023_9_news_us_old.csv
2025_1_news_us_old.csv
2025_2_news_us_old.csv
2025_3_news_us_old.csv


In [ ]:
# Check data old news recompte i schema (timestamptype)
f_old_news = my_spark.read.parquet("/TFM/old_news")
print(f_old_news.count())  # Mostra les primeres files
f_old_news.printSchema()

In [113]:
from pyspark.sql.functions import to_date, col
# revisar dates antigues
df = my_spark.read.option("header", "true").schema(schema_old_news).csv("file:///home/roser/TFM/data/old_news/1992_11_news_us_old.csv")
df = df.withColumn("pubDate", to_date(col("pubDate"), "yyyy-MM-dd"))
print(df.count())
# Filtrar les dates mal formades
df_filtered = df.filter(col("pubDate").isNotNull())
df_filtered.count()

7454


6369

In [ ]:
from pyspark.sql import functions as F
df_filtered.show()

df = df.withColumn(
    "pubDate",
    F.when(F.col("pubDate") == "1952-01-01", F.lit("1992-01-01")).otherwise(F.col("pubDate"))
)
# Calcular la data màxima i mínima
min_date = df.agg(F.min("pubDate").alias("min_date")).collect()[0]["min_date"]
max_date = df.agg(F.max("pubDate").alias("max_date")).collect()[0]["max_date"]

# Mostrar els resultats
print(f"Min Date: {min_date}")
print(f"Max Date: {max_date}")

+--------+--------------------+--------------------+----------+
|iso_code|               title|         description|   pubDate|
+--------+--------------------+--------------------+----------+
|      us|Quiet Draw for Fi...|Bobby Fischer tri...|1992-11-01|
|      us| Valvano Back on Air|JIM VALVANO, the ...|1992-11-01|
|      us|Fall Cleanup Day ...|DESPITE the threa...|1992-11-01|
|      us|If You're Thinkin...|PARK SLOPE is fam...|1992-11-01|
|      us|WHITE MEN CAN'T DRUM|What a pleasure t...|1992-11-01|
|      us|IN SHORT: NONFICTION|THE REPORTER WHO ...|1992-11-01|
|      us|Pull Up a Chair, ...|TO: Mayor David N...|1992-11-01|
|      us|Broken Hearts and...|WHERE IS HERE? St...|1992-11-01|
|      us|Who Wants Only 'T...|Recent hand-wring...|1992-11-01|
|      us|   IN SHORT: FICTION|MOSTLY HARMLESS B...|1992-11-01|
|      us|Christy Falcon, B...|Howard J. Falcon ...|1992-11-01|
|      us|   WESTCHESTER GUIDE|      INTO THE WOODS|1992-11-01|
|      us|              INSIDE|     A Fe

In [121]:
df.write.mode("append").parquet("/TFM/old_news/")